# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geopandas as gpd
import requests
import geoviews as gv
from pathlib import Path

# Import API key
from api_keys import geoapify_key



In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-37.29,100,89,0.55,RU,1703210143
1,1,bethel,41.3712,-73.4140,-1.40,53,0,2.57,US,1703209918
2,2,sant antoni de portmany,38.9807,1.3036,15.54,81,0,2.57,ES,1703210144
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.26,74,12,7.04,SH,1703210144
4,4,grytviken,-54.2811,-36.5092,3.99,88,58,5.23,GS,1703210145


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display


# Define a GeoDataFrame with the cities and their coordinates
gdf = gpd.GeoDataFrame(city_data_df, geometry=gpd.points_from_xy(city_data_df['Lng'], city_data_df['Lat']))

# Plot the map using hvplot and geoviews
map_plot = gdf.hvplot.points(
    geo=True,
    size='Humidity',
    hover_cols=['City', 'Humidity'],
    tiles='CartoLight',
    width=700,
    height=500,
    color="City",
    title='Cities Map with Humidity',
)

# Show the plot
map_plot.opts(
    tools=['hover', 'pan', 'wheel_zoom'],
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
].dropna()

ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
261,261,saint-pierre,-21.3393,55.4781,24.82,61,0,4.12,RE,1703210305
359,359,muscat,23.6139,58.5922,21.25,66,0,2.67,OM,1703210364
360,360,toliara,-23.3500,43.6667,23.64,85,0,2.25,MG,1703210364
383,383,brisas de zicatela,15.8369,-97.0419,23.31,65,0,1.09,MX,1703210379


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df  

,City,Country,Lat,Lng,Humidity,Hotel Name
261,saint-pierre,RE,-21.3393,55.4781,61,
359,muscat,OM,23.6139,58.5922,66,
360,toliara,MG,-23.3500,43.6667,85,
383,brisas de zicatela,MX,15.8369,-97.0419,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000  # You can adjust this value based on your preference
params = {
    'radius': radius,
    'type': 'hotel',
    'apiKey': geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["categories"] = "accommodation.hotel"
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
muscat - nearest hotel: البطاشي
toliara - nearest hotel: Ambary
brisas de zicatela - nearest hotel: Casa de Olas


,City,Country,Lat,Lng,Humidity,Hotel Name
261,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
359,muscat,OM,23.6139,58.5922,66,البطاشي
360,toliara,MG,-23.3500,43.6667,85,Ambary
383,brisas de zicatela,MX,15.8369,-97.0419,65,Casa de Olas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display


# Define a GeoDataFrame with the cities and their coordinates
gdf = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df['Lng'], hotel_df['Lat']))

# Plot the map using hvplot and geoviews
map_plot = gdf.hvplot.points(
    geo=True,
    size='Humidity',
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    tiles='CartoLight',
    width=700,
    height=500,
    color="City",
    title='Cities Map with Humidity',
)

# Show the plot
map_plot.opts(
    tools=['hover', 'pan', 'wheel_zoom'],
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Country,Hotel Name)